In [1]:
from openai import OpenAI

# # Load the JSON data from the file
# with open("test_files.json", "r") as file:
#     data = json.load(file)
OPENAI_API_KEY = ''
client = OpenAI(api_key=OPENAI_API_KEY)


In [23]:
import fitz
import docx2txt
import json
import os

def extract_text_from_pdf(file_path):
    with fitz.open(file_path) as doc:  # open document
        text = chr(12).join([page.get_text() for page in doc])
        return text

def extract_text_from_docx(file_path):
    text = docx2txt.process(file_path)
    return text

def extract_text(file_path):
    # Get the file extension
    file_extension = os.path.splitext(file_path)[1].lower()

    # Determine the extraction method based on the file extension
    if file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension == '.docx':
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Please provide a PDF or DOCX file.")

#code will be added in code here to filter through all docs in a folder and extract text from them instead of one at a time

file_path = "./docname.pdf"  # File name in the same folder

# Extract the file name without extension
file_name = os.path.splitext(os.path.basename(file_path))[0]

# Extract text from the file
extracted_text = extract_text(file_path)

# Create a dictionary with extracted details
file_data = {
    "name": file_name,
    "text": extracted_text
}

#print extracted text to confirm accuracy
print(json.dumps(file_data, indent=4))

{
    "name": "heat",
    "text": "Heat stress poses significant dangers to workers. Prolonged exposure to high temperatures can lead to dehydration, heat \nexhaustion, and even life-threatening heatstroke. The strain on the cardiovascular system and the increased vulnerability of \ncertain populations further highlights the need for preventive measures. This week we will provide you with some products to \nassist in controlling exposure to the summer heat. \nThe Accommodations SME\u2019s are accepting referrals! \nACCOMMODATIONS NEWS \nVolume 2/ Issue 25 (June 23, 2023) \nSummer Edition- Heat Stress Products \nCLICK HERE TO REFER \nHeat Stress Monitors ($40) \nInformation in real time!   They \ncan detect changes in body \ntemperature, heart rate, and \nsweat levels. Monitors can alert \nworkers and supervisors to \npotential heat-related risks \nbefore they escalate. \nCooling Helmets (fans $116) or \nHard Hat Inserts ($51) \nThese devices feature a cooling \nmechanism, such as a fan

In [24]:
data = [file_data]
for doc in data:
    name = doc["name"]
    text = doc["text"]
    prompt = f"""
    You will receive a text containing information about multiple accommodations. Extract each accommodation from the text with the following details:
    - Accommodation Name: The specific tool or method being used to accommodate workers. Retrieve directly from the text; do not create or alter names;;
    - Description: A detailed explanation of the accommodation.Get this info from the text directly. Retrieve directly from the text; do not create or alter description;ter names;
    - Injury Location Name: Choosing only from the list: {"Body systems", "Multiple body parts", "Cranial region, including skull", "Leg(s)", "Lower back (lumbar, sacral, coccygeal regions)", "Shoulder", "Ankle(s)", "Finger(s), fingernail(s)", "Arm(s)", "Wrist(s)", "Not Coded", "Foot (feet), except toe(s)", "Chest, including ribs, internal organs", "Pelvic region", "Upper extremities, unspecified, NEC", "Multiple trunk locations", "Multiple lower extremities locations", "Hand(s), except finger(s)", "Upper back (cervical, thoracic regions)", "Multiple back regions", "Abdomen", "Back, unspecified, NEC", "Head, unspecified, NEC", "Eye(s)", "Face", "Toe(s), toenail(s)", "Ear(s)", "Multiple head locations", "Lower extremities, unspecified, NEC", "Trunk, unspecified, NEC", "Other body parts including unclassified, NEC"} indentify the part of the body that the accommodation aims to protect or assist;
    - Industry Name: Choose the applicable options from the list: {"Agriculture, forestry, fishing and hunting", "Mining, quarrying, and oil and gas extraction", "Utilities", "Construction", "Manufacturing", "Wholesale trade", "Retail trade", "Transportation and warehousing", "Information and cultural industries", "Finance and insurance", "Real estate and rental and leasing", "Professional, scientific, and technical services", "Management of companies and enterprises", "Administrative and support, waste management, and remediation services", "Educational services", "Health care and social assistance", "Arts, entertainment, and recreation", "Accommodation and food services", "Other services (except public administration)", "Public administration"} indentify the industry in which the accommodation is used (e.g., Construction). If there is no industry specified choose  "Multiple";
    - Activity Name: Indentify the physical tasks associated with or accommodated by the accommodation.
    
    Additionally, please create a summary of the document so that any one who wishes to know what the document is about can get a brief overview. Please take ideas directly from the document only.
    
    Format the extracted data as a JSON object, with an array if multiple are mentioned in the text. Use the following structure:
    {{
      "accommodations": [
        {{
          "accommodation_name": "",
          "description": "",
          "injury_location_name": "",
          "industry_name": [],
          "activity_name": ""
        }}
      ],
      "summary": ""
    }}

    Title: {name}
    Text: {text}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to extract data from text and format it as JSON object."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    print(f"Response for '{name}':")
    print(response.choices[0].message.content)
    print("\n" + "=" * 50 + "\n")

Response for 'heat':
{
  "accommodations": [
    {
      "accommodation_name": "Heat Stress Monitors",
      "description": "Information in real time. Can detect changes in body temperature, heart rate, and sweat levels. Monitors can alert workers and supervisors to potential heat-related risks before they escalate.",
      "injury_location_name": "Body systems",
      "industry_name": ["Agriculture, forestry, fishing and hunting", "Mining, quarrying, and oil and gas extraction", "Construction", "Manufacturing", "Utilities"],
      "activity_name": "Working in high temperature environments"
    },
    {
      "accommodation_name": "Cooling Helmets",
      "description": "These devices feature a cooling mechanism, such as a fan or evaporative cooling system, to help keep the head and face cool.",
      "injury_location_name": "Cranial region, including skull",
      "industry_name": ["Construction", "Mining, quarrying, and oil and gas extraction"],
      "activity_name": "Working in hig

In [22]:
#not meant to be final code just the idea of what we would do

import datetime
#add of a verified boolean to the json object to confirm if the data is correct

#upload document to blob storage and get public viewing link returned 
url=""

verified = False
date = datetime.date.today()
toSend={
    response.choices[0].message.content,
    url,
    date,
    verified
}

#upload object to azure db in desired way based on schema


{'', '{\n  "accommodations": [\n    {\n      "accommodation_name": "Anchor Jack",\n      "description": "A telescopic tool that inserts an anchor into a pre-drilled hole and that has a sliding ‘hammering’ system. This tool reduces the amount of overhead forceful work traditionally required to do the job. It lets workers keep a more neutral body posture as they no longer need to raise their arms above their shoulders and hold them there while working on the ceiling. The extension lets workers keep a more neutral body posture. The work is conducted on the ground rather than on a ladder, scaffold, or lift.",\n      "injury_location_name": "Arm(s), Shoulder, Neck",\n      "industry_name": ["Construction"],\n      "activity_name": "Installing anchors"\n    }\n  ],\n  "summary": "The document discusses a solution to the problem of installing anchors which can cause fatigue and lead to neck, arm, and shoulder problems. The proposed solution is an Anchor Jack, a telescopic tool that can insert